In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

from hierarchicalforecast.utils import aggregate
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA,
    AutoETS,
    AutoCES,
    AutoTheta,
    SeasonalExponentialSmoothingOptimized,
    HoltWinters,
    MSTL,
    DynamicOptimizedTheta,
    GARCH,
    HistoricAverage,
    SeasonalNaive
)



c:\Users\zygog\miniconda3\envs\test\lib\site-packages\statsforecast\core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
#testing aggregate function on lacity.org website


#data = pd.read_csv(os.path.join(r"C:\Users\zygog\OneDrive\Desktop\Kaggle\Store Sales - Time Series Forecasting\Hierarchical Forecast", 'lacity.org-website-traffic.csv'), 
#                   parse_dates=['Date']).loc[:, ['Date', 'Device Category', 'Browser', 'Sessions']]

#data['Website'] = 'lacity.org'
#selected = ['Chrome', 'Firefox', 'Internet Explorer', 'Opera', 'Safari',
#       'Android Browser', 'Safari (in-app)']
#data = data.loc[data['Browser'].isin(selected), :]
#data = data.rename(columns={'Date': 'ds', 'Sessions': 'y'})

#train = data.loc[data['ds'] < '2019-01-01']
#valid = data.loc[(data['ds'] >= '2019-01-01') & (data['ds'] < '2019-02-01')]
#h = valid['ds'].nunique()

#spec = [['Website'],
#        ['Website', 'Device Category'],
#        ['Website', 'Device Category', 'Browser']]




#train_agg, S_train, tags = aggregate(train, spec)
#valid_agg, _, _ = aggregate(valid, spec)

In [2]:
train_open = pd.read_csv("output_csv/train_open.csv", parse_dates = ['date'])


data = train_open[['date','store_nbr', 'family','sales']]
data = data.rename(columns={'date': 'ds', 'sales': 'y'})
data['Company'] = 'Top_Level'


# Create a mapping for store numbers
store_nbr_mapping = {i: f's{i}' for i in range(1, 55)}

# Create a mapping for families
family_mapping = {i: f'f{i}' for i in range(33)}


# Replace store_nbr and family columns with formatted values
data['store_nbr'] = data['store_nbr'].map(store_nbr_mapping)
data['family'] = data['family'].map(family_mapping)

data.head()

,ds,store_nbr,family,y,Company
0,2013-01-01,s25,f0,0.0,Top_Level
1,2013-01-01,s25,f1,0.0,Top_Level
2,2013-01-01,s25,f2,2.0,Top_Level
3,2013-01-01,s25,f3,810.0,Top_Level
4,2013-01-01,s25,f4,0.0,Top_Level


In [11]:
#add exogenous variables and store them in a new dataframe exog_df

#exog_df = train_open[['date', 'onpromotion','dcoilwtico','day', 'day_of_week']]
#exog_df = exog_df.rename(columns={'date': 'ds'})
#exog_df.head()

#'onpromotion','dcoilwtico','day', 'day_of_week'
 # Gia na valw exogenous variables prepei na paw sto valid_data_agg kai train_data_agg kai na ta prosthesw 
 #ekei san kainoura columns. Meta na paw sto forecast kai na prosthesw to katallilo X_df


,ds,onpromotion,dcoilwtico,day,day_of_week
0,2013-01-01,0,93.14,1,1
1,2013-01-01,0,93.14,1,1
2,2013-01-01,0,93.14,1,1
3,2013-01-01,0,93.14,1,1
4,2013-01-01,0,93.14,1,1


In [14]:
data.head()

,ds,store_nbr,family,y,onpromotion,dcoilwtico,day,day_of_week,Company
0,2013-01-01,s25,f0,0.0,0,93.14,1,1,Top_Level
1,2013-01-01,s25,f1,0.0,0,93.14,1,1,Top_Level
2,2013-01-01,s25,f2,2.0,0,93.14,1,1,Top_Level
3,2013-01-01,s25,f3,810.0,0,93.14,1,1,Top_Level
4,2013-01-01,s25,f4,0.0,0,93.14,1,1,Top_Level


In [15]:
levels = [['Company'],
        ['Company', 'store_nbr'],
        ['Company', 'store_nbr', 'family']]

train_start_date = '2016-08-01'
train_end_date = '2017-07-31'
test_start_date = '2017-08-01'
test_end_date = '2017-08-15'

train_data = data[(data['ds'] >= train_start_date) & (data['ds'] <= train_end_date)]
valid_data = data[(data['ds'] >= test_start_date) & (data['ds'] <= test_end_date)]


train_data_agg, S_train1, tags1 = aggregate(train_data, levels)
valid_data_agg, S_valid1, tags2 = aggregate(valid_data, levels)



c:\Users\zygog\miniconda3\envs\test\lib\site-packages\hierarchicalforecast\utils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique_id'] = df[hier].agg('/'.join, axis=1)
c:\Users\zygog\miniconda3\envs\test\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\zygog\miniconda3\envs\test\lib\site-packages\hierarchicalforecast\utils.py:188: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the futu

In [4]:

# Create a list of models and instantiation parameters
models = [
    AutoARIMA(season_length=15, alias = 'AA15'), 
    AutoARIMA(alias = 'AA1'),
    #AutoETS(season_length=15, alias = 'AETS15'),
    #AutoETS(alias = 'AETS0'),
    #AutoCES(alias = 'ACES0'),
    AutoTheta(alias = 'ATheta0'),
    SeasonalExponentialSmoothingOptimized(season_length=1, alias = 'SESO'),
    #HoltWinters(season_length=7, alias = 'HW7', error_type='A'),   #
    #HoltWinters(alias = 'HW0'),
    MSTL(season_length=1, alias = 'MSTL'),
    DynamicOptimizedTheta(alias = 'DOT'),
    GARCH(alias = 'GARCH'),
    HistoricAverage(alias = 'HA')
    #AutoARIMA(season_length=7, alias = 'AA7'),
    #SeasonalNaive(season_length=15)
]


# Instantiate StatsForecast class as sf

sf = StatsForecast(              
    models=models,
    freq='D',            
    n_jobs=-1,
    #fallback_model = HistoricAverage(alias = 'HA')
)

sf.fit(train_data_agg)


In [24]:
p = sf.forecast(h=15, fitted=True)

In [25]:
p_fitted = sf.forecast_fitted_values()
p_fitted.head()

,ds,y,AA15,AA1,ATheta0,SESO,MSTL,DOT,"GARCH(1,1)",HA
unique_id,,,,,,,,,,
Top_Level,2016-08-01,895580.3750,894684.8125,894684.8125,835998.3125,NaN,680668.0625,895580.37500,NaN,854076.4375
Top_Level,2016-08-02,777592.8125,869067.1875,869067.1875,842170.5000,895580.3750,690184.6250,492569.21875,415356.843750,854076.4375
Top_Level,2016-08-03,779844.8125,807815.9375,807815.9375,835926.6875,889827.9375,694055.1250,521071.59375,-140467.734375,854076.4375
Top_Level,2016-08-04,622883.1250,783651.6250,783651.6250,830532.4375,884465.7500,697854.0000,546948.93750,-118823.328125,854076.4375
Top_Level,2016-08-05,704092.7500,715660.6875,715660.6875,809981.4375,871712.3750,694534.1875,554542.37500,-201974.265625,854076.4375


In [26]:
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, ERM, OptimalCombination
from hierarchicalforecast.core import HierarchicalReconciliation

reconcilers = [BottomUp(), 
               TopDown(method='forecast_proportions'),
               TopDown(method='average_proportions'),
               TopDown(method='proportion_averages'),
               #MiddleOut (middle_level='Company/store_nbr', top_down_method='forecast_proportions'),     MIDDLE OUT PRINTS MOSTLY NANS 
               #MiddleOut (middle_level='Company/store_nbr', top_down_method='average_proportions'),
               #MiddleOut (middle_level='Company/store_nbr', top_down_method='proportion_averages'),
               #MinTrace(method='ols', nonnegative=True),
               #MinTrace(method='wls_struct', nonnegative=True),
               #MinTrace(method='wls_var', nonnegative=True),
               #MinTrace(method='mint_shrink', nonnegative=True), 
               #MinTrace(method='mint_cov', nonnegative=True),
               #OptimalCombination(method='ols', nonnegative=True), 
               #OptimalCombination(method='wls_struct', nonnegative=True),
               ERM(method='closed'),
               #ERM(method='reg'),
               #ERM(method='reg_bu'),
              ]

rec_model = HierarchicalReconciliation(reconcilers=reconcilers)

p_rec = rec_model.reconcile(Y_hat_df=p, Y_df=p_fitted, S=S_train1, tags=tags1)

In [27]:
p_rec_ = p_rec.merge(valid_data_agg, on=['ds', 'unique_id'], how='left')
#p_rec_['y'] = p_rec_['y'].fillna(0)
p_rec_ = p_rec_.reset_index(drop=False)

from sklearn.metrics import mean_squared_error

rmse = dict()
for model_ in p_rec.columns[1:]:
    rmse_ = mean_squared_error(p_rec_['y'].values, p_rec_[model_].values, squared=False)
    # get only the model name
    model__ = model_.split('/')[-1]
    rmse[model__] = rmse_
pd.DataFrame(rmse, index=['RMSE']).T.sort_values('RMSE')

,RMSE
ERM_method-closed_lambda_reg-0.01,2702.833662
TopDown_method-average_proportions,2808.835183
TopDown_method-proportion_averages,2809.585090
TopDown_method-forecast_proportions,2809.585276
HA,2809.585298
BottomUp,2809.585315
AA1,2887.590571
AA15,2899.514176
DOT,2911.504325
ATheta0,2954.392446


In [28]:
p_rec_.isna().sum()

unique_id                                         0
ds                                                0
AA15                                              0
AA1                                               0
ATheta0                                           0
SESO                                              0
MSTL                                              0
DOT                                               0
GARCH(1,1)                                        0
HA                                                0
AA15/BottomUp                                     0
AA1/BottomUp                                      0
ATheta0/BottomUp                                  0
SESO/BottomUp                                     0
MSTL/BottomUp                                     0
DOT/BottomUp                                      0
GARCH(1,1)/BottomUp                               0
HA/BottomUp                                       0
AA15/TopDown_method-forecast_proportions          0
AA1/TopDown_

In [29]:
p_rec_.isna().sum()

unique_id                                         0
ds                                                0
AA15                                              0
AA1                                               0
ATheta0                                           0
SESO                                              0
MSTL                                              0
DOT                                               0
GARCH(1,1)                                        0
HA                                                0
AA15/BottomUp                                     0
AA1/BottomUp                                      0
ATheta0/BottomUp                                  0
SESO/BottomUp                                     0
MSTL/BottomUp                                     0
DOT/BottomUp                                      0
GARCH(1,1)/BottomUp                               0
HA/BottomUp                                       0
AA15/TopDown_method-forecast_proportions          0
AA1/TopDown_

In [30]:
# Check for negative values in each column except the datetime column and count them
negative_counts = p_rec_.drop(columns=['ds','unique_id']).lt(0).sum()

# Display the counts of negative values
print(negative_counts)

AA15                                                266
AA1                                                 155
ATheta0                                             235
SESO                                                  0
MSTL                                                 45
DOT                                                  15
GARCH(1,1)                                        15948
HA                                                    0
AA15/BottomUp                                       266
AA1/BottomUp                                        155
ATheta0/BottomUp                                    235
SESO/BottomUp                                         0
MSTL/BottomUp                                        45
DOT/BottomUp                                         15
GARCH(1,1)/BottomUp                               15948
HA/BottomUp                                           0
AA15/TopDown_method-forecast_proportions            266
AA1/TopDown_method-forecast_proportions         

In [31]:
# Copy the p_rec_ DataFrame to a new DataFrame
p_rec_nonnegative = p_rec_.copy()

# Set all negative values in each column to 0
p_rec_nonnegative[p_rec_nonnegative.columns.difference(['ds','unique_id'])] = p_rec_nonnegative[p_rec_nonnegative.columns.difference(['ds','unique_id'])].clip(lower=0)

# Check if negative values have been replaced with 0
negative_counts_after = p_rec_nonnegative.drop(columns=['ds','unique_id']).lt(0).sum()

# Display the counts of negative values after setting them to 0
print(negative_counts_after)

# Now, p_rec_nonnegative contains the values with negative values set to 0

AA15                                              0
AA1                                               0
ATheta0                                           0
SESO                                              0
MSTL                                              0
DOT                                               0
GARCH(1,1)                                        0
HA                                                0
AA15/BottomUp                                     0
AA1/BottomUp                                      0
ATheta0/BottomUp                                  0
SESO/BottomUp                                     0
MSTL/BottomUp                                     0
DOT/BottomUp                                      0
GARCH(1,1)/BottomUp                               0
HA/BottomUp                                       0
AA15/TopDown_method-forecast_proportions          0
AA1/TopDown_method-forecast_proportions           0
ATheta0/TopDown_method-forecast_proportions       0
SESO/TopDown

In [32]:
#Evaluate rmse for p_rec_nonnegative
from sklearn.metrics import mean_squared_error

rmse = dict()
for model_ in p_rec.columns[1:]:
    rmse_ = mean_squared_error(p_rec_['y'].values, p_rec_nonnegative[model_].values, squared=False)
    # get only the model name
    model__ = model_.split('/')[-1]
    rmse[model__] = rmse_
pd.DataFrame(rmse, index=['RMSE']).T.sort_values('RMSE')

,RMSE
ERM_method-closed_lambda_reg-0.01,2702.833646
TopDown_method-average_proportions,2808.835183
TopDown_method-proportion_averages,2809.585090
TopDown_method-forecast_proportions,2809.585276
HA,2809.585298
BottomUp,2809.585315
AA1,2887.590531
AA15,2899.499976
DOT,2911.504325
ATheta0,2954.392445


In [33]:
#Evaluate rmsle for p_rec_nonnegative
from sklearn.metrics import mean_squared_log_error

rmsle = dict()
for model_ in p_rec.columns[1:]:
    rmsle_ = mean_squared_log_error(p_rec_['y'].values, p_rec_nonnegative[model_].values, squared=False)
    # get only the model name
    model__ = model_.split('/')[-1]
    rmsle[model__] = rmsle_
pd.DataFrame(rmsle, index=['RMSLE']).T.sort_values('RMSLE')

,RMSLE
DOT,0.496825
ATheta0,0.500139
SESO,0.500581
MSTL,0.504796
ERM_method-closed_lambda_reg-0.01,0.528665
TopDown_method-forecast_proportions,0.554892
TopDown_method-proportion_averages,0.554892
HA,0.554892
BottomUp,0.554892
TopDown_method-average_proportions,0.554912


In [6]:
train_open.columns.unique()

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'year',
       'quarter', 'month', 'day', 'day_of_week', 'dcoilwtico', 'holiday_type',
       'locale', 'locale_name', 'transferred', 'transactions', 'city', 'state',
       'store_type', 'cluster'],
      dtype='object')